In [1]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.simulate import run_df_backtest, or_backtest
from datetime import datetime

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'my_stuff'

In [ ]:
import numpy as np
import plotly.graph_objects as go

from logging import getLogger
from typing import NamedTuple

from quantfreedom.helper_funcs import cart_product
from quantfreedom.indicators.tv_indicators import sma_tv
from quantfreedom.enums import CandleBodyType
from quantfreedom.strategies.strategy import Strategy

logger = getLogger("info")


class IndicatorSettingsArrays(NamedTuple):
    sma_fast_length: np.array
    sma_slow_length: np.array


class SMACrossing(Strategy):
    def __init__(
        self,
        long_short: str,
        sma_fast_length: np.array,
        sma_slow_length: np.array,
    ) -> None:
        self.long_short = long_short

        cart_arrays = cart_product(
            IndicatorSettingsArrays(
                sma_fast_length=sma_fast_length,
                sma_slow_length=sma_slow_length,
            )
        )

        sma_fast_length = cart_arrays[0].astype(np.int_)
        sma_slow_length = cart_arrays[1].astype(np.int_)

        sma_bools = sma_fast_length < sma_slow_length

        self.indicator_settings_arrays: IndicatorSettingsArrays = IndicatorSettingsArrays(
            sma_fast_length=sma_fast_length[sma_bools],
            sma_slow_length=sma_slow_length[sma_bools],
        )

        if long_short == "long":
            self.set_entries_exits_array = self.long_set_entries_exits_array
            self.log_indicator_settings = self.long_log_indicator_settings
            self.entry_message = self.long_entry_message
        else:
            self.set_entries_exits_array = self.short_set_entries_exits_array
            self.log_indicator_settings = self.short_log_indicator_settings
            self.entry_message = self.short_entry_message

    #######################################################
    #######################################################
    #######################################################
    ##################      Long     ######################
    ##################      Long     ######################
    ##################      Long     ######################
    #######################################################
    #######################################################
    #######################################################

    def long_set_entries_exits_array(
        self,
        candles: np.array,
        ind_set_index: int,
    ):
        try:
            closes = candles[:, CandleBodyType.Close]

            # sma fast
            self.sma_fast_length = self.indicator_settings_arrays.sma_fast_length[ind_set_index]
            sma_fast = sma_tv(
                source=closes,
                length=self.sma_fast_length,
            )

            self.sma_fast = np.around(sma_fast, 2)

            self.prev_sma_fast = np.roll(self.sma_fast, 1)
            self.prev_sma_fast[0] = np.nan
            logger.info(f"Created sma_fast sma_fast_length= {self.sma_fast_length}")

            # sma slow
            self.sma_slow_length = self.indicator_settings_arrays.sma_slow_length[ind_set_index]
            sma_slow = sma_tv(
                source=closes,
                length=self.sma_slow_length,
            )

            self.sma_slow = np.around(sma_slow, 2)

            self.prev_sma_slow = np.roll(self.sma_slow, 1)
            self.prev_sma_slow[0] = np.nan
            logger.info(f"Created sma_slow sma_slow_length= {self.sma_slow_length}")

            # Entries
            self.entries = (self.prev_sma_fast < self.prev_sma_slow) & (self.sma_fast > self.sma_slow)
            self.cross_above_signal = np.where(self.entries, sma_fast, np.nan)
            logger.debug("Created entries and cross above signals")

            # Exits
            exits = (self.prev_sma_fast > self.prev_sma_slow) & (self.sma_fast < self.sma_slow)
            self.cross_below_signal = np.where(exits, sma_fast, np.nan)
            self.exit_prices = np.roll(np.where(exits, closes, np.nan), 1)
            logger.debug("Created exit prices and cross below signals")

        except Exception as e:
            logger.error(f"Exception long_set_entries_exits_array -> {e}")
            raise Exception(f"Exception long_set_entries_exits_array -> {e}")

    def long_log_indicator_settings(
        self,
        ind_set_index: int,
    ):
        logger.info(
            f"Indicator Settings Index= {ind_set_index}\
            \nsma_fast_length= {self.sma_fast_length}\
            \nsma_slow_length= {self.sma_slow_length}"
        )

    def long_entry_message(
        self,
        bar_index: int,
    ):
        logger.info("\n\n")
        logger.info("Entry time!!! prev_sma_fast < prev_sma_slow & sma_fast > sma_slow")
        logger.info(
            f"{self.prev_sma_fast[bar_index]} < {self.prev_sma_slow[bar_index]} & {self.sma_fast[bar_index]} > {self.sma_slow[bar_index]}"
        )

    #######################################################
    #######################################################
    #######################################################
    ##################      short    ######################
    ##################      short    ######################
    ##################      short    ######################
    #######################################################
    #######################################################
    #######################################################

    def short_set_entries_exits_array(
        self,
        candles: np.array,
        ind_set_index: int,
    ):
        pass

    def short_log_indicator_settings(
        self,
        ind_set_index: int,
    ):
        pass

    def short_entry_message(
        self,
        bar_index: int,
    ):
        pass

    #######################################################
    #######################################################
    #######################################################
    ##################      Live     ######################
    ##################      Live     ######################
    ##################      Live     ######################
    #######################################################
    #######################################################
    #######################################################

    def live_set_indicator(
        self,
        closes: np.array,
    ):
        pass

    def live_evaluate(
        self,
        candles: np.array,
    ):
        pass

    #######################################################
    #######################################################
    #######################################################
    ##################      Plot     ######################
    ##################      Plot     ######################
    ##################      Plot     ######################
    #######################################################
    #######################################################
    #######################################################

    def plot_signals(
        self,
        candles: np.array,
    ):
        timestamp = candles[:, CandleBodyType.Timestamp]
        datetimes = timestamp.astype("datetime64[ms]")
        open = candles[:, CandleBodyType.Open]
        high = candles[:, CandleBodyType.High]
        low = candles[:, CandleBodyType.Low]
        close = candles[:, CandleBodyType.Close]

        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=datetimes,
                    open=open,
                    high=high,
                    low=low,
                    close=close,
                    name="Candles",
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.sma_fast,
                    name="SMA Fast",
                    line_color="lightblue",
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.sma_slow,
                    name="SMA Slow",
                    line_color="yellow",
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.cross_above_signal,
                    mode="markers",
                    name="Entries",
                    marker=dict(
                        size=12,
                        symbol="circle",
                        color="#00F6FF",
                        line=dict(
                            width=1,
                            color="DarkSlateGrey",
                        ),
                    ),
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.cross_below_signal,
                    mode="markers",
                    name="Exits",
                    marker=dict(
                        size=12,
                        symbol="triangle-up",
                        color="#FF7B00",
                        line=dict(
                            width=1,
                            color="DarkSlateGrey",
                        ),
                    ),
                ),
            ]
        )
        fig.update_layout(
            height=800,
            xaxis_rangeslider_visible=False,
            title=dict(
                x=0.5,
                text="Signals",
                xanchor="center",
                font=dict(
                    size=40,
                ),
            ),
        )
        fig.show()

    def get_strategy_plot_filename(
        self,
        candles: np.array,
    ):
        pass

In [ ]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

In [ ]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="30m",
    candles_to_dl=3000,
    # since_datetime=datetime(2023, 11, 14, 19),
    # until_datetime=datetime(2023, 11, 16),
)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([0]),
    sl_based_on_add_pct=np.array([0.05, 0.15]),
    sl_based_on_lookback=np.array([30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1, 2]),
    trail_sl_when_pct=np.array([1, 3]),
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="market",
    tp_strategy_type=TakeProfitStrategyType.Nothing,
    trail_sl_bool=True,
    z_or_e_type=None,
)

strategy = SMACrossing(
    long_short="long",
    sma_fast_length=np.arange(20, 61, 20),
    sma_slow_length=np.arange(30, 91, 30),
)

In [ ]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=False,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 8
Total combinations of settings to test: 48
Total candles: 3,000
Total candles to test: 144,000


In [ ]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(5)

ind_set_idx  dos_index  total_trades  wins  losses  gains_pct  win_rate  \
2             0          2          24.0     9      15    434.199    37.500   
4             0          4          38.0    20      18    212.743    52.632   
0             0          0          42.0    20      22    193.558    47.619   
6             0          6          23.0     9      14    336.110    39.130   
21            2          5           9.0     7       2    181.346    77.778   

    qf_score  fees_paid  total_pnl  ending_eq  
2      0.893    917.707   4341.992   5341.992  
4      0.871    472.895   2127.428   3127.428  
0      0.867    492.568   1935.580   2935.580  
6      0.809    827.190   3361.105   4361.105  
21     0.806     53.044   1813.460   2813.460

In [ ]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=2,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 0.0,
    sl_based_on_add_pct = 0.0005,
    sl_based_on_lookback = 30,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.02,
    trail_sl_when_pct = 0.01,
)


In [ ]:
order_records_df

ind_set_idx  or_set_idx  bar_idx      timestamp            datetime  \
0              0           2       58  1697288400000 2023-10-14 13:00:00   
1              0           2       70  1697310000000 2023-10-14 19:00:00   
2              0           2       72  1697313600000 2023-10-14 20:00:00   
3              0           2       96  1697356800000 2023-10-15 08:00:00   
4              0           2      139  1697434200000 2023-10-16 05:30:00   
..           ...         ...      ...            ...                 ...   
276            0           2     2954  1702501200000 2023-12-13 21:00:00   
277            0           2     2956  1702504800000 2023-12-13 22:00:00   
278            0           2     2984  1702555200000 2023-12-14 12:00:00   
279            0           2     2985  1702557000000 2023-12-14 12:30:00   
280            0           2     2986  1702558800000 2023-12-14 13:00:00   

       order_status    equity  available_balance  cash_borrowed  cash_used  \
0       EntryFilled  1000.000            948.715       4230.130     51.285   
1       EntryFilled  1000.000            782.553      24031.404    217.447   
2    StopLossFilled   933.835            933.835            NaN        NaN   
3       EntryFilled   933.835            879.836       5164.723     53.999   
4          MovedTSL       NaN                NaN            NaN        NaN   
..              ...       ...                ...            ...        ...   
276        MovedTSL       NaN                NaN            NaN        NaN   
277        MovedTSL       NaN                NaN            NaN        NaN   
278        MovedTSL       NaN                NaN            NaN        NaN   
279        MovedTSL       NaN                NaN            NaN        NaN   
280  StopLossFilled  5341.992           5341.992            NaN        NaN   

     ...  entry_size_usd  entry_price  exit_price  position_size_asset  \
0    ...        4281.415      26882.3         NaN                0.159   
1    ...       15737.306      26858.4         NaN                0.745   
2    ...             NaN          NaN     26815.2                  NaN   
3    ...        5218.722      26872.7         NaN                0.194   
4    ...             NaN          NaN         NaN                  NaN   
..   ...             ...          ...         ...                  ...   
276  ...             NaN          NaN         NaN                  NaN   
277  ...             NaN          NaN         NaN                  NaN   
278  ...             NaN          NaN         NaN                  NaN   
279  ...             NaN          NaN         NaN                  NaN   
280  ...             NaN          NaN     42223.1                  NaN   

     position_size_usd  realized_pnl  sl_pct  sl_price  tp_pct  tp_price  
0             4281.415           NaN   0.600   26726.1     NaN       NaN  
1            20018.721           NaN   0.200   26815.2     NaN       NaN  
2                  NaN       -66.165     NaN       NaN     NaN       NaN  
3             5218.722           NaN   0.400   26760.7     NaN       NaN  
4                  NaN           NaN   0.012   27183.9     NaN       NaN  
..                 ...           ...     ...       ...     ...       ...  
276                NaN           NaN   0.009   41895.4     NaN       NaN  
277                NaN           NaN   0.015   42124.1     NaN       NaN  
278                NaN           NaN   0.016   42188.7     NaN       NaN  
279                NaN           NaN   0.017   42223.1     NaN       NaN  
280                NaN       210.822     NaN       NaN     NaN       NaN  

[281 rows x 27 columns]